# CNN implementation

In [ ]:
import keras
import numpy as np
import glob

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.utils import plot_model

class Model:
    
    def __init__(self, dataset_path):
        self.img_width = 150
        self.img_height = 150
        self.dataset_path = dataset_path
        self.dataset_path_train = "/".join([dataset_path, "train"])
        self.dataset_path_dev = "/".join([dataset_path, "dev"])
        self.epochs = 50
        self.batch_size = 8
        self.len_train = len(glob.glob(self.dataset_path_train + "/*/*.jpg"))
        self.len_dev = len(glob.glob(self.dataset_path_dev + "/*/*.jpg"))
        self.n_train_label = len(glob.glob(self.dataset_path_train + "/*/"))
        self.n_dev_label = len(glob.glob(self.dataset_path_dev + "/*/"))
    
    def run_training(self):
        
        ## Build model architecture
        if K.image_data_format() == 'channels_first':
            input_shape = (3, self.img_width, self.img_height)
        else:
            input_shape = (self.img_width, self.img_height, 3)
        
        print("input_shape", input_shape)
        print("n train label", self.n_train_label)
        print("n dev label", self.n_dev_label)
        
        self.model = Sequential()
        self.model.add(Conv2D(32, (3, 3), input_shape=input_shape))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Conv2D(32, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Conv2D(64, (3, 3)))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))

        self.model.add(Flatten())
        self.model.add(Dense(self.n_train_label))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.5))
        self.model.add(Activation('softmax'))

        self.model.compile(loss='categorical_crossentropy',
                      optimizer='rmsprop',
                      metrics=['accuracy'])
        
        # plot_model(self.model, to_file='model.png')
        self.model.summary()

        ## Define data generation
        train_datagen = ImageDataGenerator(
            rescale=1. / 255,
            horizontal_flip=True)

        test_datagen = ImageDataGenerator(rescale=1. / 255)

        train_generator = train_datagen.flow_from_directory(
            self.dataset_path_train,
            target_size=(self.img_width, self.img_height),
            batch_size=self.batch_size,
            class_mode='categorical')

        validation_generator = test_datagen.flow_from_directory(
            self.dataset_path_dev,
            target_size=(self.img_width, self.img_height),
            batch_size=self.batch_size,
            class_mode='categorical')
    
        
        self.history = self.model.fit_generator(
            train_generator,
            epochs=self.epochs,
            steps_per_epoch=self.len_train//self.batch_size,
            validation_data=validation_generator, 
            validation_steps=self.len_dev//self.batch_size)
        

        self.model.save_weights('first_try.h5')

    def plot_model(self):
        # summarize history for accuracy
        plt.plot(self.history.history['acc'])
        plt.plot(self.history.history['val_acc'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()
        
        # summarize history for loss
        plt.plot(self.history.history['loss'])
        plt.plot(self.history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.show()
        
modeler = Model("dataset/train_classes")
modeler.run_training()

In [ ]:
modeler.model